In [223]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import re
import glob
import os
import pandas as pd
from os import system
import csv
from nipype.interfaces.freesurfer import MRIConvert
import subprocess
#import nbpackage.mynotebook

dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
reconlog = "/autofs/cluster/animal/scan_data/leads/recon/recon.log"
recondir = "/autofs/cluster/animal/scan_data/leads/recon/"
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]

# wipe clean execute.recon.list ----------------- either end of file or here?
open(recondir+'execute.recon.list', 'w').close()
pswd= ''

In [224]:
def parse_scaninfo(origpath):
    with open(origpath+'/scan.info', 'r') as in_file:
        editline = line.split()
        with open(origpath+'/scaninfo.csv', 'w') as result:
            wr = csv.writer(result, dialect='excel')
            wr.writerow(editline)
            result.close()
    in_file.close()

In [225]:
for substring in folders:
    subjectdir = recondir+substring
    #if substring not in open(reconlog).read():
    if "_" not in substring:  
        print('yes')

yes


In [226]:
%%time 
# APPROX Wall time: 46.5 s PER SUBJECT

# load in new version of excel doc from LONI with exclusion criteria (add comments to ours somehow)

for substring in folders:
    subjectdir = recondir+substring
    #if substring not in open(reconlog).read():
    if "_" not in substring:    
        print('unpacking ' + substring + " ----------------------- ")
        os.makedirs(recondir+substring+'/mri/orig/')
        
        # check number of date folders; check number of session folders
        MPRAGE_list = glob.glob(dicomdir+substring + '/Accelerated_Sagittal_MPRAGE/*/*/', recursive=True)

        if len(MPRAGE_list) > 1:
            print("There are multiple MPRAGEs for "+substring+". Unpacking only the first; check others.")
        
        # for now, proceed with unpacking only the first valid run (given multiple)
        # check for errors and record RUN#
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_list[0], subjectdir, subjectdir)
        system(cmdstring)

        parse_scaninfo(subjectdir)
        scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
        check = scan_info.iloc[0,2] # first row (only row); second col (validity)
        
        #convert from dicom's MPRAGE to mgz (allow for more than one)
        if check != 'ok':
            print(MPRAGE_list[0]+" IS AN INVALID RUN (I.E. NEEDS TO BE OMMITTED FROM ANALYSIS). SKIPPING THE UNPACKING.")
        else:
            # make list of all files within MPRAGE_list[0]
            dicomlist = os.listdir(MPRAGE_list[0])
            with open(subjectdir+'/dicomlist.txt', 'w') as f:
                for item in dicomlist:
                    f.write("%s%s\n" % (subjectdir, '/'+item))
                    
            mc = MRIConvert()
            mc.inputs.in_file = MPRAGE_list[0]+os.listdir(MPRAGE_list[0])[0] # first file
            #mc.inputs.sdcm_list = subjectdir+'/dicomlist.txt' # no need for this
            mc.inputs.out_file = subjectdir+'/mri/orig/001.mgz'
            mc.inputs.out_type = 'mgz'
            system(mc.cmdline)

            #obtain date: must change folder names in both leadsir/recon and /leadsdir/LEADS
            datestring = scan_info.iloc[0,7]
            date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', datestring).group()[4:]

            #rename both folders
            os.rename(recondir+substring, recondir+substring+'_'+date)
            os.rename(dicomdir+substring, dicomdir+substring+'_'+date)
            
            # create execute.recon.list to launch recon list
            with open(recondir+'execute.recon.list', "a") as myfile:
                myfile.write(substring+'_'+date)

            ##add substring to log
            with open(reconlog, "a") as myfile:
                myfile.write(substring+'_'+date)
    


unpackingLDS0110021 ----------------------- 
CPU times: user 23 ms, sys: 21 ms, total: 44.1 ms
Wall time: 47.2 s


In [236]:
import shlex
# works but not I just need to do this through launchpad
origcall = "ssh rje@launchpad 'bash -s' < /autofs/cluster/animal/scan_data/leads/recon/execute.recon.sh"
listcall = shlex.split(origcall)
print(listcall)
#subprocess.call(recondir+'execute.recon.sh', shell=True)

['ssh', 'rje@launchpad', 'bash -s', '<', '/autofs/cluster/animal/scan_data/leads/recon/execute.recon.sh']


In [235]:
recondir+'execute.recon.sh'

'/autofs/cluster/animal/scan_data/leads/recon/execute.recon.sh'

In [109]:
        # the scan.info file only has ONE run (because every run's dicoms are in separate folders)
#         unpack = UnpackSDICOMDir()
#         unpack.inputs.source_dir = MPRAGE_list[0]
#         unpack.inputs.output_dir = subjectdir
#         unpack.inputs.run_info = (1, 'mprage', 'nii', 'struct')
#         unpack.inputs.dir_structure = 'generic'
#         unpack.cmdline
subjectdir

'/autofs/cluster/animal/scan_data/leads/recon/LDS0110021'

In [230]:
# only thing left is to run the bash script execute.recon on launchpad

#from paramiko import SSHClient
import paramiko
import sys

hostname= 'launchpad'
username="rje11"
command = 'ls'

client = paramiko.Transport((hostname, port))
client.connent(username=username, password=pswd)

stdout_data = []
stderr_data = []
session = client.open_channel(kind='session')
session.exec_command(command)


print


# # run the script for recon (will read in the execute.recon.list to execute.recon:
# cmd = "ssh rje11@launchpad 'bash -s' < /autofs/cluster/animal/scan_data/leads/recon/execute.recon.sh"
# system(cmd)







ssh rje11@launchpad 'bash -s' < /autofs/cluster/animal/scan_data/leads/recon/execute.recon.sh


256

In [164]:
scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
datestring = scan_info.iloc[0,7]
date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', datestring).group()[4:]
date

# use re to extract date from this string (first 8 digits after raw_) # is this the same for non-MPRAGES?

'20181016'

In [127]:
subjectdir

'/autofs/cluster/animal/scan_data/leads/recon/LDS0110021'

In [128]:
#/autofs/cluster/animal/scan_data/leads/recon/LDS0110021/mri/orig/
MPRAGE_list[0]+os.listdir(MPRAGE_list[0])[0]
#/cluster/animal/scan_data/leads/LEADS/LDS0110021/Accelerated_Sagittal_MPRAGE/*/*/`ls /cluster/animal/scan_data/leads/LEADS/LDS0110021/Accelerated_Sagittal_MPRAGE/*/*/ | head -n 1`

'/cluster/animal/scan_data/leads/LEADS/LDS0110021/Accelerated_Sagittal_MPRAGE/2018-10-16_09_44_47.0/S736558/LEADS_LDS0110021_MR_Accelerated_Sagittal_MPRAGE__br_raw_20181016131243397_163_S736558_I1060269.dcm'

In [ ]:
#         mri_convert /cluster/animal/scan_data/leads/LEADS/$substring/Accelerated_Sagittal_MPRAGE/
# */*/`ls /cluster/animal/scan_data/leads/LEADS/$substring/Accelerated_Sagittal_MPRAGE/*/*/ | head -n 1` /au
# tofs/cluster/animal/scan_data/leads/recon/$substring/mri/orig/001.mgz

In [213]:
# bash file is created separately (in terminal) for now
# create bash file to eventually launch with ssh
# reconbash = recondir+"execute.recon.sh" 
# line0 = '#!/bin/bash;\n'
# line1 = 'setenv FREESURFER_HOME /usr/local/freesurfer/stable6_0_0;\n'
# line2 = 'source $FREESURFER_HOME/SetUpFreeSurfer.csh;\n'
# line3 = 'setenv SUBJECTS_DIR %s;\n' % recondir 
# line4 = 'echo HELLO' # here make a for loop that calls another script (list)
# line5 = # read line from file
# line6 = 'foreach s (*)'
# line7 = 'pbsubmit "recon-all all -subjid ${s}"'
# line8 = 'sleep 10'
# line9 = 'end'
# file = open(reconbash, "w")
# file.writelines([line0, line1, line2, line3, line4])
# file.close()

# cmd = subprocess.Popen(['bash',reconbash])
#system(cmd)


SyntaxError: invalid syntax (<ipython-input-213-edef1c1b2e2d>, line 8)